In [ ]:
import cv2

In [ ]:
# Pfade zu positiven und negativen Bildern
positive_images_path = 'positive_images/'
negative_images_path = 'negative_images/'

# Pfad zum speichern des trainierten Modells
cascade_model_path = 'trained_model.xml'

In [ ]:
# Trainieren des Cascaden-Modells
cv2.samples_create_samples_from_info(
    'info.dat', positive_images_path, negative_images_path,
    img_size=(50, 50), num_samples=1000, bg_color=(0, 0, 0),
    invert=False, max_horizontal_shift=0, max_vertical_shift=0,
    max_rotation_angle=0, max_scale_factor=0
)

cv2.traincascade(cv2.CascadeClassifier_create(), 'info.dat',
                 'cascade_model.xml', numPos=1000, numNeg=500,
                 numStages=20, minHitRate=0.999, maxFalseAlarmRate=0.5,
                 stages=None, features=None)

In [1]:
# import the necessary packages
import os
import zipfile
import matplotlib.pyplot as plt
import cv2
import glob
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.applications import ResNet50, ResNet101, ResNet152, InceptionV3, VGG16
import numpy as np
from PIL import Image
from PIL import Image, ImageOps
from keras.models import load_model



cascade_speedsign = cv2.CascadeClassifier('cascade/cascade.xml')

classes = { 0:'Dominik',  
            1:'Unknown'}

# Function to detect faces
def detect_faces(image):
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = cascade_speedsign.detectMultiScale(rgb, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))
    return faces

# Initialize webcam
cap = cv2.VideoCapture(0)


# Check if the webcam is available
if not cap.isOpened():
    print("No webcam found or access denied.")
else:
    # Infinite loop for face recognition
    while True:
        # Capture frame from the webcam
        ret, frame = cap.read()
        # Detect faces
        faces = detect_faces(frame)

        # Make predictions for each detected face
        for (x, y, w, h) in faces:
            
            # Expand the frame of the face
            val = 40
            x = max(0, x - val)
            y = max(0, y - val)
            w = min(frame.shape[1], val+x+w) - x
            h = min(frame.shape[0], val+y+h) - y
            face_image = frame[y:y+h, x:x+w]
            
            #face_image = np.expand_dims(face_image, axis=0)
            face_image = img_to_array(face_image)
            # normalize the image
            face_image = cv2.normalize(face_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            face_image = array_to_img(face_image)
            face_image = face_image.resize((224, 224))
            face_image = np.expand_dims(face_image, axis=0)
            face_image = np.array(face_image)

            # Perform face classification
            # get the label from the highest prediction
            predictions = np.argmax(model.predict(face_image), axis=1)[0]        
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, classes[predictions], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Face Recognition', frame)

        # Exit the loop by pressing 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam stream
cap.release()
cv2.destroyAllWindows()

OSError: No file or directory found at models/Resnet152_model_adam.h5